In [2]:
import numpy as np
import os
from astroquery.jplhorizons import Horizons

In [3]:
# Paths to directories containing data
home_path = os.path.expanduser('~/')
path_67p = home_path+'Documents/year1/shape_modelling/67p/'

In [9]:
# Constants
km2au = 1.496e8
c = 3e5

In [10]:
### Function to create a file in the 'Mikko' format required for shape modelling 

def mikkoWrite(filename,day_list,int_list,sunx,suny,sunz,earthx,earthy,earthz, min_dist):
# Code to write to lightcurve file in Mikko format
# Additional argument to limit files to points beyond min_dist from the sun

    counter = 0
    fout = open(filename, 'w+')
    sameday = []
    for i, day in enumerate(day_list[:-1]):
        if (np.sqrt(sunx[i]**2+suny[i]**2+sunz[i]**2) >= min_dist):
            newday = True
            sameday.append(day)
            if abs(day-day_list[i+1])<0.5:
                newday = False
            if newday==False:
                if day not in sameday:
                    sameday.append(day)
            else:
                fout.write('%i 1\n'%(len(sameday)))
                for time in sameday:
                    loc = day_list.index(time)
                    fout.write('%.6f %.6e %.10f %.8f %.8f %.8f %.8f %.8f\n'%(time+2400000.5,int_list[loc],sunx[loc], suny[loc], sunz[loc], earthx[loc], earthy[loc], earthz[loc]))
                counter = counter+1
                sameday = []
    fout.write('1 1\n')
    fout.write('%.6f %.6e %.10f %.8f %.8f %.8f %.8f %.8f\n'%(day_list[-1]+2400000.5,int_list[-1],sunx[-1], suny[-1], sunz[-1], earthx[-1], earthy[-1], earthz[-1]))
    counter = counter+1
    fout.close()
    return counter

In [11]:
# Read LSST sim output

lsst_file_name = path_67p+'LSST_out_LC_67P.txt'
lsst_file = open(lsst_file_name, 'r')

lsst_mjd =[]; lsst_delta=[]
lsst_ra = []; lsst_dec = []
lsst_sunx = []; lsst_suny = []; lsst_sunz = []
lsst_phase_ang = [];
lines_lsst = lsst_file.readlines()
for line in lines_lsst[25:]:
    dat = line.split()
    lsst_mjd.append(float(dat[2]))
    lsst_delta.append(float(dat[3]))
    lsst_ra.append(float(dat[5]))
    lsst_dec.append(float(dat[7]))
    lsst_sunx.append(float(dat[9]))
    lsst_suny.append(float(dat[10]))
    lsst_sunz.append(float(dat[11]))
    lsst_phase_ang.append(float(dat[21]))

lsst_delta = np.array(lsst_delta)/km2au
lsst_sunx = np.array(lsst_sunx)/km2au
lsst_suny = np.array(lsst_suny)/km2au
lsst_sunz = np.array(lsst_sunz)/km2au

In [12]:
# Check it got everything
print(len(lsst_mjd))

678


In [13]:
mjd = np.array(lsst_mjd)
sun_x = lsst_sunx
sun_y = lsst_suny
sun_z = lsst_sunz

- - - - - - - - - - - - -

In [27]:
# LSST output only has obj-Sun geom: also need Earth-obj
# Also want list of solar phase angles, to compute abs mag from phase function
# ^Phase angles in LSST output

# Splitting this into 2 because Horizons wouldn't let me do all 678 epochs at once :(
obj = Horizons(id='90000696', location='500',epochs=mjd[:500])

In [28]:
earth_x = []; earth_y=[]; earth_z=[];
vec = obj.vectors(refplane='earth')  # LSST output in equatorial ref frame!!!
for i in range(len(vec['x'])):
    earth_x.append(float(vec['x'][i]))
    earth_y.append(float(vec['y'][i]))
    earth_z.append(float(vec['z'][i]))

In [29]:
# Check output looks sensible
vec

targetname,datetime_jd,datetime_str,M1,x,k1,y,z,vx,vy,vz,lighttime,range,range_rate
---,d,---,mag,AU,---,AU,AU,AU / d,AU / d,AU / d,d,AU,AU / d
str25,float64,str30,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
67P/Churyumov-Gerasimenko,2459953.863937,A.D. 2023-Jan-09 08:44:04.1568,13.1,-3.444482371454907,9.0,-2.0153218155794,-0.7824936038575984,0.01361521546668406,-0.001640161614941021,-0.001023053609964404,0.02348746242662225,4.066728054307531,-0.01052231245788887
67P/Churyumov-Gerasimenko,2459956.868077,A.D. 2023-Jan-12 08:50:01.8528,13.1,-3.40395502934091,9.0,-2.019033115617043,-0.7850424525508048,0.01335825588027279,-0.0008328510046677227,-0.0006747805678868234,0.0233031223252092,4.034810555161236,-0.01072159808861264
67P/Churyumov-Gerasimenko,2459957.822871,A.D. 2023-Jan-13 07:44:56.0544,13.1,-3.391243793174929,9.0,-2.019707085350671,-0.7856344182491888,0.01326711280976805,-0.0005791513265564079,-0.0005653072657361154,0.02324384071155479,4.024546261940707,-0.0107784009087213
67P/Churyumov-Gerasimenko,2459964.842786,A.D. 2023-Jan-20 08:13:36.7104,13.1,-3.300815343852066,9.0,-2.017347222055183,-0.7868281284840544,0.01245379431047052,0.001235273828283649,0.0002185167014543807,0.02279989299093424,3.947679096927296,-0.01108792928393342
67P/Churyumov-Gerasimenko,2459966.871729,A.D. 2023-Jan-22 08:55:17.3856,13.1,-3.275830694466186,9.0,-2.014328084827037,-0.7861629347424759,0.01217090610427359,0.001738902614021462,0.0004363793981198722,0.02266965365805568,3.925128855476299,-0.01113737028412288
67P/Churyumov-Gerasimenko,2459966.876288,A.D. 2023-Jan-22 09:01:51.2832,13.1,-3.27577520880826,9.0,-2.014320154620278,-0.7861609441857158,0.01217024671962196,0.001740021155361504,0.0004368633081598101,0.02266936040337532,3.925078080002388,-0.01113745921781113
67P/Churyumov-Gerasimenko,2459970.830448,A.D. 2023-Jan-26 07:55:50.7072,13.1,-3.228832861105711,9.0,-2.00555375025243,-0.7836176940127041,0.01156088228071457,0.002685654418529836,0.0008458298592107309,0.02241441696974867,3.880935992834392,-0.011176991382028
67P/Churyumov-Gerasimenko,2459970.830859,A.D. 2023-Jan-26 07:56:26.2176,13.1,-3.228828109596892,9.0,-2.005552646428814,-0.7836173463681377,0.0115608152568932,0.002685750070184072,0.0008458712014704912,0.02241439043849895,3.880931399090906,-0.01117699163366914


In [30]:
# Now get the remaining 655-500 vectors
obj = Horizons(id='90000696', location='500', epochs=mjd[500:])

In [31]:
vec = obj.vectors(refplane='earth')  
for i in range(len(vec['x'])):
    earth_x.append(float(vec['x'][i]))
    earth_y.append(float(vec['y'][i]))
    earth_z.append(float(vec['z'][i]))

In [32]:
# Check this didn't miss anything or double anything
len(earth_x)

678

In [33]:
# Light time correct the MJDs:
delta = lsst_delta # in km
#for i in range(len(earth_x)):
#    delta.append(np.sqrt(earth_x[i]**2+earth_y[i]**2+earth_z[i]**2))
#delta=np.array(delta)
mjd_ltc = np.zeros_like(mjd)

mjd_ltc = mjd-(((delta*km2au)/c)/(3600*24))
mjd_ltc = np.ndarray.tolist(mjd_ltc)

In [50]:
for i,time in enumerate(mjd_ltc):
    print(time, mjd[i])

59953.34046686406 59953.363937
59956.34479107718 59956.368077
59957.29964427473 59957.322871
59964.320002923756 59964.342786
59966.3490760892 59966.371729
59966.353635384934 59966.376288
59970.30805011133 59970.330448
59970.30846113815 59970.330859
59970.323103093804 59970.3455
59970.323931147774 59970.346328
59972.297099372634 59972.319369
59973.35521357923 59973.377415
59977.31767781625 59977.339625
59978.30851600644 59978.3304
59979.344438877495 59979.366257
59980.278004965185 59980.299764
59984.24211719965 59984.263628
59984.245833433575 59984.267344
59984.25742616275 59984.278936
59984.25825021452 59984.27976
59984.30237198136 59984.323879
59990.31762575978 59990.338768
59990.333675713955 59990.354817
59990.349401646774 59990.370542
59991.25495492466 59991.276042
59991.271517907946 59991.292604
59992.31625583932 59992.337281
59992.33249678522 59992.353521
59994.31207227407 59994.332983
59994.31248729774 59994.333398
59994.32763415996 59994.348544
59994.32805218372 59994.348962
599

In [34]:
sun_xyz = np.array([sun_x, sun_y, sun_z])
print(sun_xyz[:,0])
earth_xyz = np.array([earth_x, earth_y, earth_z])
print(earth_xyz[:,0])

[-3.75625676 -1.15950396 -0.41150245]
[-3.44448237 -2.01532182 -0.7824936 ]


In [35]:
namestr = path_67p+'test.txt'
mikkoWrite(namestr,mjd_ltc,np.ones_like(mjd),sun_x,sun_y,sun_z,earth_x,earth_y,earth_z,0.0)
# Go and type the number output rom this cell into the top of the file you've just created
# This is the number of lightcurves in total

326

In [39]:
# Checking the phase angles: should make this into a function but too lazy
alpha_list_calc = []
for i in range(len(earth_xyz[1])):
    alpha_list_calc.append((360/(2.*np.pi))*np.arccos(np.dot(sun_xyz[:,i],earth_xyz[:,i])/(np.linalg.norm(sun_xyz[:,i])*np.linalg.norm(earth_xyz[:,i]))))

In [40]:
# This is just to grab ephemeris data to check phase angles
# Will also use this to grab rh values for completion's sake
obj = Horizons(id='90000696', location='500', epochs=mjd[:500])
ephem_table = obj.ephemerides()

/Users/s1523386/anaconda3/lib/python3.8/site-packages/astroquery/jplhorizons/core.py:604: UserWarning: The uri used in this query is very long and might have been truncated. The results of the query might be compromised. If you queried a list of epochs, consider querying a range.
  warnings.warn(('The uri used in this query is very long '


In [41]:
ephem_table

targetname,datetime_str,datetime_jd,M1,solar_presence,k1,flags,RA,DEC,RA_app,DEC_app,RA_rate,DEC_rate,AZ,EL,AZ_rate,EL_rate,sat_X,sat_Y,sat_PANG,siderealtime,airmass,magextinct,Tmag,Nmag,illumination,illum_defect,sat_sep,sat_vis,ang_width,PDObsLon,PDObsLat,PDSunLon,PDSunLat,SubSol_ang,SubSol_dist,NPole_ang,NPole_dist,EclLon,EclLat,r,r_rate,delta,delta_rate,lighttime,vel_sun,vel_obs,elong,elongFlag,alpha,lunar_elong,lunar_illum,sat_alpha,sunTargetPA,velocityPA,OrbPlaneAng,constellation,TDB-UT,ObsEclLon,ObsEclLat,NPole_RA,NPole_DEC,GlxLon,GlxLat,solartime,earth_lighttime,RA_3sigma,DEC_3sigma,SMAA_3sigma,SMIA_3sigma,Theta_3sigma,Area_3sigma,RSS_3sigma,r_3sigma,r_rate_3sigma,SBand_3sigma,XBand_3sigma,DoppDelay_3sigma,true_anom,hour_angle,alpha_true,PABLon,PABLat
---,---,d,mag,---,---,---,deg,deg,deg,deg,arcsec / h,arcsec / h,deg,deg,arcsec / min,arcsec / min,arcsec,arcsec,deg,---,---,mag,mag,mag,%,arcsec,arcsec,---,arcsec,deg,deg,deg,deg,deg,arcsec,deg,arcsec,deg,deg,AU,km / s,AU,km / s,min,km / s,km / s,deg,---,deg,deg,%,deg,deg,deg,deg,---,s,deg,deg,deg,deg,deg,deg,---,min,arcsec,arcsec,arcsec,arcsec,deg,arcsec2,arcsec,km,km / s,Hz,Hz,s,deg,---,deg,deg,deg
str25,str24,float64,float64,str1,float64,str1,float64,float64,float64,float64,float64,float64,int64,int64,int64,int64,float64,float64,float64,int64,int64,int64,float64,int64,float64,float64,float64,str1,float64,int64,int64,int64,int64,float64,float64,int64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,str2,float64,float64,float64,float64,float64,float64,float64,str3,float64,float64,float64,int64,int64,float64,float64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,float64,float64,float64
67P/Churyumov-Gerasimenko,2023-Jan-09 08:44:04.157,2459953.863937002,13.1,,9.0,,210.33,-11.09304,210.63351,-11.20313,17.52865,-6.54362,--,--,--,--,-265903.97,39295.153,263.256,--,999,--,21.518,--,98.51574,2e-05,275032.6,*,0.001356,--,--,--,--,110.3,0.0,--,--,198.0968,1.2131,3.952678956373,8.8807927,4.06656779493586,-18.2186996,33.82061306,13.8660151,23.8097204,76.3979,/L,13.9941,77.5,94.7719,89.61,290.367,295.164,-0.89176,Vir,69.184123,212.4108533,1.1778817,--,--,329.155298,48.189203,--,0.0,0.045,0.077,0.081,0.037,-69.213,0.0095503,0.089,55.9448,3.4e-06,0.02,0.09,0.000373,139.5795,--,13.9955,205.0941,1.2055
67P/Churyumov-Gerasimenko,2023-Jan-12 08:50:01.853,2459956.868077,13.1,,9.0,,210.67266,-11.21882,210.97679,-11.32868,16.0473,-5.97696,--,--,--,--,-276518.59,37202.391,262.024,--,999,--,21.516,--,98.49611,2.1e-05,284753.8,*,0.001367,--,--,--,--,110.17,0.0,--,--,198.3631,1.196,3.968046201996,8.8321875,4.03465192167608,-18.5637112,33.55517683,13.8032144,23.2027281,79.0983,/L,14.0871,42.0,75.8416,86.8167,290.235,295.123,-0.91299,Vir,69.184209,212.7702321,1.1749518,--,--,329.53989,47.93826,--,0.0,0.046,0.078,0.082,0.038,-69.187,0.0097267,0.09,55.9597,3.4e-06,0.02,0.07,0.000373,139.8462,--,14.0882,205.4067,1.1955
67P/Churyumov-Gerasimenko,2023-Jan-13 07:44:56.054,2459957.822871,13.1,,9.0,,210.77521,-11.2564,211.07951,-11.36619,15.56523,-5.79432,--,--,--,--,-279933.33,36497.907,261.641,--,999,--,21.515,--,98.49118,2.1e-05,287867.5,*,0.00137,--,--,--,--,110.13,0.0,--,--,198.4473,1.1906,3.972912673262,8.8167782,4.02438816432538,-18.662049,33.46981577,13.7833688,23.0132746,79.9632,/L,14.1104,30.5,67.5056,85.9285,290.194,295.11,-0.9193,Vir,69.184236,212.8777235,1.1740324,--,--,329.654175,47.862921,--,0.0,0.046,0.078,0.083,0.038,-69.179,0.0097845,0.091,55.9529,3.4e-06,0.02,0.07,0.000373,139.9305,--,14.1114,205.5025,1.1924
67P/Churyumov-Gerasimenko,2023-Jan-20 08:13:36.710,2459964.842786,13.1,,9.0,,211.43051,-11.49622,211.73633,-11.60561,11.84834,-4.41031,--,--,--,--,-305628.75,30766.86,258.943,--,999,--,21.508,--,98.4756,2.1e-05,311129.5,*,0.001397,--,--,--,--,109.85,0.0,--,--,199.0602,1.151,4.008432289785,8.7040549,3.94752521669402,-19.1978762,32.83056612,13.6391203,21.6714687,86.4249,/L

In [42]:
alpha_list = []; rh_list = []
for i,row in enumerate(ephem_table['alpha_true']):
    alpha_list.append(float(row))
    rh_list.append(float(ephem_table['r'][i]))

In [43]:
obj = Horizons(id='90000696', location='geocentric', epochs=mjd[500:])
ephem_table = obj.ephemerides()

In [44]:
# Append the rest of the values to the lists
for i,row in enumerate(ephem_table['alpha_true']):
    alpha_list.append(float(row))
    rh_list.append(float(ephem_table['r'][i]))

In [45]:
for i in range(len(alpha_list)):
    print(alpha_list[i], lsst_phase_ang[i])

13.9955 13.993685
14.0882 14.086378
14.1114 14.109652
14.1844 14.182674
14.1723 14.170457
14.1723 14.170399
14.1037 14.101938
14.1037 14.101926
14.1033 14.101524
14.1033 14.101501
14.0463 14.044583
14.0094 14.007557
13.8316 13.829852
13.7773 13.775489
13.7161 13.714225
13.6572 13.65551
13.3668 13.365123
13.3664 13.364812
13.3655 13.363841
13.3654 13.363772
13.3618 13.360051
12.7925 12.790664
12.7908 12.788885
12.7891 12.787139
12.6898 12.688142
12.688 12.686249
12.569 12.567086
12.5671 12.565145
12.3285 12.326588
12.3284 12.326535
12.3265 12.324593
12.3265 12.324539
11.9402 11.938493
11.9402 11.938435
11.9379 11.936064
11.9378 11.936005
11.1933 11.19135
11.1907 11.188692
11.0331 11.031215
11.0305 11.028539
10.5438 10.542192
10.5387 10.537024
10.1746 10.172745
9.9953 9.993589
8.3857 8.384018
8.3821 8.380298
8.3788 8.376931
7.9549 7.953266
7.9513 7.949652
6.5689 6.567276
6.5632 6.561435
6.5585 6.556755
5.592 5.59033
5.5918 5.590119
5.5879 5.586163
5.5876 5.585951
5.5787 5.576914
5.5747 5

In [ ]:
####-----Go and use mag_from_phase_func.py to get shifted absolute magnitudes-----#####

In [46]:
# Read in shifted magnitudes:
shifted_mags_file = path_67p+'67P_mag_list_ALL.txt'
shifted_mags_list = []
magf=open(shifted_mags_file, 'r')
lines = magf.readlines()
for line in lines:
    data = line.split()
    shifted_mags_list.append(float(data[0]))
#shifted_mags_list

In [48]:
# Final step: read in intensities as calculated from shifts from Matlab script
# Obvs if you're using this script to create Mikko file in the first instance, you won't have these
intensities_file = path_67p+'67P_int_list_ALL.txt'
intensities_list = []
intf = open(intensities_file, 'r')
lines = intf.readlines()
for line in lines:
    data = line.split()
    intensities_list.append(float(data[0]))
#intensities_list

In [49]:
# Create Mikko file with real intensities (not dummy)
print(len(intensities_list))

namestr_fin = path_67p+'67P_LC_NU.dat'
mikkoWrite(namestr_fin,mjd_ltc,intensities_list,sun_x,sun_y,sun_z,earth_x,earth_y,earth_z,0.0)
# Go and type the number output rom this cell into the top of the file you've just created
# This is the number of lightcurves in total

678


326

In [50]:
# Also making standard lightcurve output with dates, magnitudes and 0.01 mag. uncertainty

corr_fac = 16    # from mag_from_phase_func py script

fout = open(path_67p+'67p_20230109_I11_R.dat', 'w+')
for i,day in enumerate(mjd_ltc):
    #magnitude = corr_fac -2.5*np.log10(intensities_list[i])
    magnitude = shifted_mags_list[i]
    fout.write('%.6f %.4f 0.01 %.8f %.8f %.8f %.8f %.8f %.8f %.8f %.8f\n'%(day+2400000.5, magnitude, sun_x[i], sun_y[i], sun_z[i], rh_list[i], earth_x[i], earth_y[i], earth_z[i], delta[i]))
fout.close()